In [4]:
import optuna
from lightgbm import LGBMRegressor, early_stopping as lgb_early_stopping
from xgboost import XGBRegressor, callback as xgb_callback
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
import numpy as np 
import pandas as pd


In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v13.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v13.csv")



In [8]:
# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]
# df = challenge_set_updated.sample(frac=0.001)

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']



In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Handle categorical columns by Label Encoding
label_encoder = LabelEncoder()

# Specify the columns that are categorical
categorical_cols = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_season', 
                    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction', 
                    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight']

# Convert these categorical columns into numerical form using Label Encoding
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Now, you can proceed with training LightGBM with this processed data


In [5]:
# Global variable to hold the trained model pipeline
global_model_pipeline = None

# Define the objective function
def objective(trial):
    global global_model_pipeline
    
    # Sample hyperparameters for LightGBM
    lgb_params = {
        'n_estimators': trial.suggest_int('lgb_n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('lgb_learning_rate', 0.001, 0.5),
        'max_depth': trial.suggest_int('lgb_max_depth', 3, 15),
        'num_leaves': trial.suggest_int('lgb_num_leaves', 31, 1024),
        'reg_alpha': trial.suggest_float('lgb_reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('lgb_reg_lambda', 0, 10),
    }

    # Sample hyperparameters for XGBoost
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.001, 0.5),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 15),
        'subsample': trial.suggest_float('xgb_subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('xgb_reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('xgb_reg_lambda', 0, 10),
    }

    # Sample hyperparameters for CatBoost
    cat_params = {
        'n_estimators': trial.suggest_int('cat_n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('cat_learning_rate', 0.001, 0.5),
        'depth': trial.suggest_int('cat_depth', 3, 15),
        'l2_leaf_reg': trial.suggest_float('cat_l2_leaf_reg', 1, 10),
    }

    # Initialize the models with sampled hyperparameters
    lgb_model = LGBMRegressor(**lgb_params)
    xgb_model = XGBRegressor(**xgb_params, objective='reg:squarederror')
    cat_model = CatBoostRegressor(**cat_params, verbose=0)

    # Ensemble the models using VotingRegressor
    ensemble_model = VotingRegressor(
        estimators=[
            ('lgb', lgb_model),
            ('xgb', xgb_model),
            ('cat', cat_model)
        ]
    )

    # Create a pipeline with data scaling and the ensemble model
    model_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('ensemble', ensemble_model)
    ])

    # Cross-validation for better evaluation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Metrics
    rmse_list = []
    r2_list = []
    mae_list = []

    for train_index, valid_index in kf.split(X):
        # Use iloc to select by position
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        # Train the ensemble model with early stopping for each model
        lgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                      callbacks=[lgb_early_stopping(stopping_rounds=50)])
        
        xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
                      callbacks=[xgb_callback.early_stop(50)], verbose=False)
        
        cat_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=50)

        # Fit the ensemble
        model_pipeline.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model_pipeline.predict(X_valid)
        
        # Calculate metrics for this fold
        rmse = mean_squared_error(y_valid, y_pred, squared=False)
        r2 = r2_score(y_valid, y_pred)
        mae = mean_absolute_error(y_valid, y_pred)

        rmse_list.append(rmse)
        r2_list.append(r2)
        mae_list.append(mae)

    # Compute average metrics
    avg_rmse = sum(rmse_list) / len(rmse_list)
    avg_r2 = sum(r2_list) / len(r2_list)
    avg_mae = sum(mae_list) / len(mae_list)

    # Save the trained model pipeline globally
    global_model_pipeline = model_pipeline

    return avg_rmse  # Optimize RMSE (could also return R², MAE for report

In [6]:
# Create an Optuna study object and optimize
study = optuna.create_study(direction='minimize')  # 'minimize' for minimizing RMSE
study.optimize(objective, n_trials=100)

# Print the best trial and its hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value (RMSE): {trial.value}")
print(f"  Params: {trial.params}")


[I 2024-10-11 13:48:41,244] A new study created in memory with name: no-name-7b7eddd3-7b72-4d0a-af1b-f7de86157c51


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020814 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17909
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 420
[LightGBM] [Info] Start training from score 79542.054059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-11 13:49:17,965] Trial 0 finished with value: 2630.520763947488 and parameters: {'lgb_n_estimators': 237, 'lgb_learning_rate': 0.20428599846981163, 'lgb_max_depth': 9, 'lgb_num_leaves': 234, 'xgb_n_estimators': 54, 'xgb_learning_rate': 0.2613707302908038, 'xgb_max_depth': 8, 'xgb_subsample': 0.7063625591208229, 'cat_n_estimators': 296, 'cat_learning_rate': 0.17976884207191882, 'cat_depth': 10}. Best is trial 0 with value: 2630.520763947488.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019167 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17909
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 420
[LightGBM] [Info] Start training from score 79542.054059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-11 13:50:12,702] Trial 1 finished with value: 2866.70030245074 and parameters: {'lgb_n_estimators': 185, 'lgb_learning_rate': 0.14025325110342132, 'lgb_max_depth': 10, 'lgb_num_leaves': 252, 'xgb_n_estimators': 227, 'xgb_learning_rate': 0.1268604654250812, 'xgb_max_depth': 7, 'xgb_subsample': 0.8144833433202276, 'cat_n_estimators': 96, 'cat_learning_rate': 0.08900815074901136, 'cat_depth': 6}. Best is trial 0 with value: 2630.520763947488.


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019466 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17909
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 420
[LightGBM] [Info] Start training from score 79542.054059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-11 15:38:16,768] Trial 98 finished with value: 2553.8047387168954 and parameters: {'lgb_n_estimators': 247, 'lgb_learning_rate': 0.15431727800813194, 'lgb_max_depth': 8, 'lgb_num_leaves': 243, 'xgb_n_estimators': 265, 'xgb_learning_rate': 0.19594082263421128, 'xgb_max_depth': 9, 'xgb_subsample': 0.8529994979327856, 'cat_n_estimators': 288, 'cat_learning_rate': 0.2640223426899314, 'cat_depth': 10}. Best is trial 83 with value: 2541.291925644221.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022819 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17909
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 420
[LightGBM] [Info] Start training from score 79542.054059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

/home/carolima/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-11 15:39:27,344] Trial 99 finished with value: 2565.6972828807247 and parameters: {'lgb_n_estimators': 249, 'lgb_learning_rate': 0.1530433194715539, 'lgb_max_depth': 8, 'lgb_num_leaves': 246, 'xgb_n_estimators': 271, 'xgb_learning_rate': 0.133917326884803, 'xgb_max_depth': 9, 'xgb_subsample': 0.9176535061655904, 'cat_n_estimators': 272, 'cat_learning_rate': 0.2629644676442357, 'cat_depth': 10}. Best is trial 83 with value: 2541.291925644221.


Best trial:
  Value (RMSE): 2541.291925644221
  Params: {'lgb_n_estimators': 231, 'lgb_learning_rate': 0.18451745341989234, 'lgb_max_depth': 8, 'lgb_num_leaves': 248, 'xgb_n_estimators': 283, 'xgb_learning_rate': 0.15914346022447923, 'xgb_max_depth': 10, 'xgb_subsample': 0.7939895757166775, 'cat_n_estimators': 286, 'cat_learning_rate': 0.2948681435460218, 'cat_depth': 10}


In [9]:
# Use the best hyperparameters to create models
best_params = study.best_trial.params

# Initialize models with the best parameters
lgb_model = LGBMRegressor(
    n_estimators=best_params['lgb_n_estimators'],
    learning_rate=best_params['lgb_learning_rate'],
    max_depth=best_params['lgb_max_depth'],
    num_leaves=best_params['lgb_num_leaves']
)

xgb_model = XGBRegressor(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    subsample=best_params['xgb_subsample'],
    objective='reg:squarederror'
)

cat_model = CatBoostRegressor(
    n_estimators=best_params['cat_n_estimators'],
    learning_rate=best_params['cat_learning_rate'],
    depth=best_params['cat_depth'],
    verbose=0
)

# Ensemble the models
ensemble_model = VotingRegressor(
    estimators=[
        ('lgb', lgb_model),
        ('xgb', xgb_model),
        ('cat', cat_model)
    ]
)

    # Split the dataset (use your own dataset instead of X, y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


# Train the ensemble model on the entire dataset
ensemble_model.fit(X_train, y_train)



[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019708 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17909
[LightGBM] [Info] Number of data points in the train set: 295210, number of used features: 420
[LightGBM] [Info] Start training from score 79542.054059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

VotingRegressor(estimators=[('lgb',
                             LGBMRegressor(learning_rate=0.18451745341989234,
                                           max_depth=8, n_estimators=231,
                                           num_leaves=248)),
                            ('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_t...
                                          learning_rate=0.15914346022447923,
                                          max_bin=None, max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=10,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=283,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=None, ...)),
                            ('cat',
                             <catboost.core.CatBoostRegressor object at 0x7d5e9518dba0>)])

In [10]:

# Make predictions
y_pred = ensemble_model.predict(X_valid)

r2 = r2_score(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))


print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")


Best Model Performance - R^2 Score: 0.9977, RMSE: 2541.2919


In [11]:
import joblib

# Save the ensemble model to a file
model_filename = 'ensemble_model.pkl'
joblib.dump(ensemble_model, model_filename)

print(f"Model saved to {model_filename}")


Model saved to ensemble_model.pkl


In [12]:
from sklearn.preprocessing import LabelEncoder

submission_set_features = submission_set_updated.iloc[:,:-1]

# Now you can use the model to make predictions
submission_set['tow'] = ensemble_model.predict(submission_set_features)
print(submission_set)


        flight_id        date                          callsign  adep  \
0       248753821  2022-01-01  3b3de0f3ad0ee192513995c02f7bf7cf  LTFJ   
1       248753822  2022-01-01  e06dd03d4a879ca37d9e18c1bd7cad16  EBBR   
2       248754498  2022-01-01  2d3b1c962c78c4ebeef11bcd51b9e94c  KMIA   
3       248757623  2022-01-01  81564432d3ee97c4bdf4cd8f006753dc  EGCN   
4       248763603  2022-01-01  84be079d7e660db105d91f600b4b3d59  EIDW   
...           ...         ...                               ...   ...   
105954  258066302  2022-12-31  2d3b4446c4d05a25196a9d52cab936fb  LTFJ   
105955  258068609  2022-12-31  253fd692ed441fac523081471c067772  LOWW   
105956  258068876  2022-12-31  c9fca302ca2e28acab0eb0bb1b46f11b  LTFM   
105957  258064675  2022-12-31  00f96ad0e382476649574ba044c764fc  EHAM   
105958  258058370  2022-12-31  5f0c222c7f7ceff3fbe75c854cce74c9  UBBB   

                     name_adep country_code_adep  ades          name_ades  \
0       Istanbul Sabiha Gokcen                

In [13]:
import os
from datetime import datetime

# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Define a timestamp for the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")

# Assuming submission_set is a DataFrame, save it to CSV
submission_set.to_csv(submission_file, index=False)

print(f"Submission saved to {submission_file}")


Submission saved to submissions/submission_20241011_171902.csv
